_____
# About The Cab_Pinot Notebook
_______

* This notebook initially takes in a fully preprocessed csv with all observations and no accounting for imbalance. Another dataframe is then created that slices out observations where the value is either `Cabernet Sauvignon` or `Pinot Noir` in the `varietal` column.

* Target and predictor variables are created from this dataframe, a baseline model is prepared and quickly analyzed, the df is split into training and testing sets, vectorized and then fit for modeling. 

* I fit three different models here based on the performance of the 20 class multiclassifier on this dataset. Random Forest, Logistic Regression and Multinomial Naive Bayes. 

* After this initial modeling I then run all three models after applying Synthetic Minority Oversampling Technique (SMOTE). 

**Summary of Results**

SMOTE had a largely insiginificant effect on overall F1 Score. The only model that benefitted from SMOTE was Logistic Regression which saw no increase in its test score but saw it go from slightly overfit to neither under or overfit. 

____
## Table of Contents
____


* [Baseline Model](#baseline)
* [Random Forest Model](#random)
* [Multinomial Naive Bayes](#mnb)
* [Logistic Regression](#log_reg)
* [Boostrapped Multinomial Naive Bayes](#bootstrap)

# Imports

In [265]:
import pandas as pd
import numpy as np

# Importing Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# SKLearn Libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# SKLearn Model Libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

# SKLearn Metric Libraries
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, classification_report
from sklearn.metrics import roc_curve, plot_roc_curve, auc, f1_score
from sklearn import metrics

# Other Libraries
from collections import Counter
from imblearn.over_sampling import SMOTE
from random import sample

from pathlib import Path
import pickle

_______
# Read in Data

In [302]:
# wine_df = pd.read_csv('/Users/jamesopacich/Documents/dsi/projects/capstone_archive/data/preprocessed_ready_4_model.csv') 
wine_df = pd.read_csv('../data/003_preprocessed_all_data.csv')

In [303]:
wine_df.head(1)

,varietal,description,color,parsed,parsed_w_stops
0,Riesling,pineapple rind lemon pith blossom start bit op...,white,pineapple rind lemon pith and orange blossom s...,pineapple rind lemon pith blossom start bit op...


In [304]:
wine_df.isna().sum()

varietal          0
description       6
color             0
parsed            0
parsed_w_stops    6
dtype: int64

In [305]:
wine_df.dropna(inplace = True)

In [306]:
wine_df.isna().sum()

varietal          0
description       0
color             0
parsed            0
parsed_w_stops    0
dtype: int64

In [307]:
cab_pinot = wine_df[(wine_df['varietal'] == 'Cabernet Sauvignon') | (wine_df['varietal'] == 'Pinot Noir')]
cab_pinot['varietal'].value_counts()

Cabernet Sauvignon    15197
Pinot Noir            12192
Name: varietal, dtype: int64

In [308]:
cab_pinot = cab_pinot[['varietal','parsed_w_stops']]
cab_pinot.reset_index(inplace = True)
cab_pinot

,index,varietal,parsed_w_stops
0,25930,Pinot Noir,regular comes tannic rustic earthy herbal char...
1,25931,Cabernet Sauvignon,supple plum envelopes oaky structure supported...
2,25932,Cabernet Sauvignon,slightly reduced chalky tannic explosion accen...
3,25936,Pinot Noir,sleek hint widely available neutral staved ind...
4,25938,Pinot Noir,intermingle robust floor designated hails bodi...
...,...,...,...
27384,73653,Cabernet Sauvignon,result bodied firm structure leather plum espr...
27385,73655,Cabernet Sauvignon,bodied firm tannins soften vanilla cassis
27386,73656,Pinot Noir,acidity hinting toasty future moment aging
27387,73657,Cabernet Sauvignon,cranberry clove anisette espresso bean hint cr...


In [309]:
cab_pinot.to_csv('../data/cab_pinot.csv')

__________
# Create Target and Predictor Variables
_________

In [310]:
X = cab_pinot['parsed_w_stops']
# Creating Multi-Class Targets
y = cab_pinot['varietal']

In [311]:
# Binarizing binary targets
y = y.map({'Pinot Noir': 0, 'Cabernet Sauvignon': 1})

In [312]:
print(f'Feature Variables Shape: {X.shape}')
print(f'Target Variables Shape: {y.shape}')

Feature Variables Shape: (27389,)
Target Variables Shape: (27389,)


<a id='baseline'></a>
________
# Baseline Model
* Randomly guessing a varietal would choose the positive class, Cabernet Sauvignon, 55% of the time and the negative class, Pinot Noir 45% of the time. 
_____

In [313]:
y.value_counts(normalize = True)

1    0.554858
0    0.445142
Name: varietal, dtype: float64

# Train / Test / Split

In [314]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = .33, 
                                                   stratify = y, 
                                                   random_state = 42)

# Vectorizing

{'clf': RandomForestClassifier(min_samples_split=5, n_estimators=50), 'vect__max_df': 0.8, 'vect__max_features': 500, 'vect__ngram_range': (1, 3), 'vect__stop_words': None}


In [315]:
# Instantiate CountVectorizer()
cvec = CountVectorizer(max_df = .8, max_features = 500, 
                       ngram_range = (1,3))

# fit CountVectorizer()
cvec.fit(X_train)

CountVectorizer(max_df=0.8, max_features=500, ngram_range=(1, 3))

In [316]:
# Transform the corpus on training data
X_train = cvec.transform(X_train)
X_test = cvec.transform(X_test)

<a id='random'></a>
_____
# Random Forest
____

In [1]:
forest = RandomForestClassifier(min_samples_split = 5, n_estimators = 50)
forest.fit(X_train, y_train)
train_preds = forest.predict(X_train)
preds = forest.predict(X_test)

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
# Testing Score
score = metrics.f1_score(y_test, preds, average = 'binary')

# Training Score
score_train = metrics.f1_score(y_train, train_preds, average = 'binary')

print(f'Training F1 Score: {round(score_train * 100, 2)}')
print(f'Test F1 Score: {round(score * 100, 2)}')



<a id='mnb'></a>
________
# Multinomial Naive Bayes
_______

In [319]:
mnb = MultinomialNB(alpha = 1.0)
mnb.fit(X_train, y_train)
train_preds = mnb.predict(X_train)
preds = mnb.predict(X_test)

In [320]:
# Testing Score
score = metrics.f1_score(y_test, preds, pos_label = 1, average = 'binary')

# Training Score
score_train = metrics.f1_score(y_train, train_preds, pos_label = 1, average = 'binary')

print(f'Training F1 Score: {round(score_train * 100, 2)}')
print(f'Test F1 Score: {round(score * 100, 2)}')



Training F1 Score: 81.05
Test F1 Score: 80.52


_____
### Pickling MNB for use in Streamlit
_____

In [321]:
# saving the model in pickle format and storing it as 'mnb_red_clf.pkl'
pickle_out = open('../06_streamlit/mnb_red_clf.pkl', mode = 'wb')

# sending mnb model to 'pickle_out' 
pickle.dump(mnb, pickle_out)
pickle_out.close()

<a id='log_reg'></a>
________
# Logisitic Regression
_______

In [322]:
log_reg = LogisticRegression(solver = 'liblinear')
log_reg.fit(X_train, y_train)
train_preds = log_reg.predict(X_train)
preds = log_reg.predict(X_test)

In [323]:
# Testing Score
score = metrics.f1_score(y_test, preds, pos_label = 1, average = 'binary')

# Training Score
score_train = metrics.f1_score(y_train, train_preds, pos_label = 1, average = 'binary')

print(f'Training F1 Score: {round(score_train * 100, 2)}')
print(f'Test F1 Score: {round(score * 100, 2)}')

Training F1 Score: 82.88
Test F1 Score: 81.0


<a id='bootstrap'></a>
______
# Bootstrap Imbalanced Classes
_____

#### SMOTE 

"SMOTE stands for **Synthetic Minority Oversampling Technique.** This is a statistical technique for **increasing the number of cases in your dataset in a balanced way.** The module works by generating new instances from existing minority cases that you supply as input. This implementation of SMOTE does not change the number of majority cases.

The **new instances are not just copies** of existing minority cases; instead, the algorithm takes samples of the feature space for each target class and its nearest neighbors, and generates new examples that combine features of the target case with features of its neighbors. This approach increases the features available to each class and makes the samples more general.

SMOTE takes the entire dataset as an input, but it increases the percentage of only the minority cases. For example, suppose you have an imbalanced dataset where just 1% of the cases have the target value A (the minority class), and 99% of the cases have the value B. To increase the percentage of minority cases to twice the previous percentage, you would enter 200 for SMOTE percentage in the module's properties."

* https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/smote#:~:text=SMOTE%20stands%20for%20Synthetic%20Minority,dataset%20in%20a%20balanced%20way.&text=SMOTE%20takes%20the%20entire%20dataset,of%20only%20the%20minority%20cases.

In [2]:
def smote_oversample(x, y):

    # Instantiate Oversampler
    oversample = SMOTE()
    
    # fit oversampler
    x, y = oversample.fit_resample(x, y)
    
    # Counter creates a dict that summarizes the distribution
    counter = Counter(y)

    for key, value in counter.items():
        
        # create variable with percentage of samples of each variable 
        percent = round(value / len(y_train) * 100, 2)
        
    # create df of target labels
    ovr_sampled = pd.DataFrame(counter.keys(), columns = ['target_label'])
    # append number of samples to df
    ovr_sampled['num_samples'] = counter.values()
    # append percentage of samples to df
    ovr_sampled['percent'] = percent

    plt.bar(ovr_sampled['target_label'], ovr_sampled['percent'])
    
    plt.title('Class Labels as Percentage of Samples')
    plt.xlabel('Class')
    plt.ylabel('Percentage of Labels')
    plt.xticks(rotation = 90)
    plt.show(); 
    
    # return df of sample distributions
    print(ovr_sampled)
    
    return x, y

In [3]:
XS_train, ys_train = smote_oversample(X_train, y_train)

NameError: name 'X_train' is not defined

_____
### MNB with SMOTE
_____

In [326]:
smote_mnb = MultinomialNB(alpha = 1.0)
smote_mnb.fit(XS_train, ys_train)
train_preds = smote_mnb.predict(XS_train)
preds = smote_mnb.predict(X_test)

In [327]:
# Testing Score
score = metrics.f1_score(y_test, preds, average = 'binary')

# Training Score
score_train = metrics.f1_score(ys_train, train_preds, average = 'binary')

print(f'Training F1 Score: {round(score_train * 100, 2)}')
print(f'Test F1 Score: {round(score * 100, 2)}')



Training F1 Score: 78.71
Test F1 Score: 79.94


In [328]:
smote_log_reg = LogisticRegression(solver = 'liblinear')
smote_log_reg.fit(XS_train, ys_train)
train_preds = smote_log_reg.predict(XS_train)
preds = smote_log_reg.predict(X_test)

In [329]:
# Testing Score
score = metrics.f1_score(y_test, preds, average = 'binary')

# Training Score
score_train = metrics.f1_score(ys_train, train_preds, average = 'binary')

print(f'Training F1 Score: {round(score_train * 100, 2)}')
print(f'Test F1 Score: {round(score * 100, 2)}')



Training F1 Score: 80.37
Test F1 Score: 79.06


In [330]:
smote_forest = RandomForestClassifier(min_samples_split = 5, n_estimators = 50)
smote_forest.fit(XS_train, ys_train)
train_preds = forest.predict(XS_train)
preds = forest.predict(X_test)

In [331]:
# Testing Score
score = metrics.f1_score(y_test, preds, average = 'binary')

# Training Score
score_train = metrics.f1_score(ys_train, train_preds, average = 'binary')

print(f'Training F1 Score: {round(score_train * 100, 2)}')
print(f'Test F1 Score: {round(score * 100, 2)}')



Training F1 Score: 95.6
Test F1 Score: 80.91
